# Поиск токсичных комментариев (с BERT)

<h2> (Тема №13: Машинное обучение для текстов) <a class="tocSkip"> </h2>

<a name="1"></a>
## 1. Содержание

[1. Содержание](#1)

[2. Описание проекта](#2)

*    [2.1. Цель проекта](#21)
*    [2.2. Задачи проекта](#22)
*    [2.3. Описание данных](#23)
*    [2.4. План работы](#24)

[3. Подготовка данных](#3)

*    [3.1. Изучение данных](#31)
*    [3.2. Сэмплирование данных](#32)
*    [3.3. Разделение сэмпла на выборки](#33)
*    [3.4. Получение эмбеддингов (с *BERT*)](#34)
*    [3.5. Вывод](#35)

[4. Обучение и тестирование моделей](#4)

*    [4.1. Функция для обучения моделей](#41)
*    [4.2. `LogisticRegression`](#42)
*    [4.3. `RandomForestClassifier`](#43)
*    [4.4. `LGBMClassifier`](#44)
*    [4.5. `CatBoostClassifier`](#45)
*    [4.6. Сравнение моделей](#46)
*    [4.7. Тестирование лучшей модели](#47)
*    [4.8. Вывод](#48)

[5. Общий вывод](#5)

<a name="2"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
## 2. Описание проекта

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучим модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Построим модель со значением метрики качества *F1* не меньше 0.75. 

<a name="21"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
### 2.1. Цель проекта

Провести исследование с целью построения модели машинного обучения, которая поможет классифицировать комментарии на позитивные и негативные.

Результаты исследования позволят магазину искать токсичные комментарии и отправлять их на модерацию.

<a name="22"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
### 2.2. Задачи проекта

Решим поставленную в проекте задачу **на базе нейронной сети *BERT***.

1. Изучить данные.
2. Сэмплировать данные.
3. Разделить сэмплы на выборки.
4. Получить эмбеддинги.
5. Построить и обучить модели.
6. Протестировать лучшую модель.
7. Написать общий вывод.

Построим модель со значением метрики качества *F1* не меньше 0.75.

<a name="23"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
### 2.3. Описание данных

В нашем распоряжении набор данных с разметкой о токсичности правок.

Данные находятся в файле `toxic_comments.csv`.

Столбец `text` в нём содержит текст комментария, а `toxic` — целевой признак.

<a name="24"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
### 2.4. План работы

1. Изучим данные.
2. Сэмплируем данные: возьмём из датасета выборку из 1000 случайных элементов.
3. Разделим сэмплированную выборку на обучающую и тестовую выборки в соотношении 4:1.
4. Переведём текст комментариев в векторные представления (эмбеддинги) на базе нейронной сети *BERT*. Используем библиотеки  `torch` и `transformers`. Лемматизация для *BERT* не требуется.
7. Обучим 4 модели: `LogisticRegression`, `RandomForestClassifier`, `LGBMClassifier` и `CatBoostClassifier` с различными гиперпараметрами.
8. Сравненим модели.
9. Протестируем лучшую модель и напишем вывод.

<a name="3"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
## 3. Подготовка данных

In [1]:
import numpy as np
import pandas as pd
import torch
import transformers
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score, \
GridSearchCV, KFold, train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tqdm import notebook

RANDOM_STATE = 12345
TEST_SIZE = 0.2

<a name="31"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
### 3.1. Изучение данных

Загрузим данные.

In [2]:
try:
    data = pd.read_csv(r'C:/Users/lorad/OneDrive/Documents/Моя папка/Data Science/Курсы/'
                        'Яндекс.Практикум. Специалист по Data Science/Проектная работа/'
                        '13. Машинное обучение для текстов/toxic_comments.csv')
except:
    try:
        data = pd.read_csv(r'D:/Юлия/Data Science/Курсы/'
                            'Яндекс.Практикум. Специалист по Data Science/Проектная работа/'
                            '13. Машинное обучение для текстов/toxic_comments.csv')
    except:
        data = pd.read_csv('https://docs.google.com/spreadsheets/d/e/'
        '2PACX-1vQ59JmNL2DruMdFkZoOga-GFUBVFTSgDnVt4Pt7SErYdQQ7hHrTSzRaBHYMhpwa_K4xlnKs_8zrW6di/'
                           'pub?gid=1802044232&single=true&output=csv')

In [3]:
display(data.sample(5))

,Unnamed: 0,text,toxic
140947,141099,"Article proposal \nTB, would you mind commenti...",0
59738,59804,Please see the media as this is a blatantly ho...,0
147353,147509,AndyTheGrump either you are referring to the g...,0
49616,49671,Woohoo! \n\nI'm a sysop! Thanks for letting me...,0
141544,141697,Thanks for your reply:\n \n1- Who has raised t...,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Датасет имеет большой размер.

In [5]:
print(data.shape)

(159292, 3)


In [6]:
data.describe()

,Unnamed: 0,toxic
count,159292.000000,159292.000000
mean,79725.697242,0.101612
std,46028.837471,0.302139
min,0.000000,0.000000
25%,39872.750000,0.000000
50%,79721.500000,0.000000
75%,119573.250000,0.000000
max,159450.000000,1.000000


Проверим наличие явных дубликатов.

In [7]:
data.duplicated().sum()

0

Подсчитаем количество классов в таргете.

In [8]:
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

Наблюдается сильный дисбаланс классов в таргете.

<a name="32"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
### 3.2. Сэмплирование данных

Мы будем решать поставленную в проекте задачу **на базе нейронной сети *BERT*** (от англ. *Bidirectional Encoder Representations from Transformers*, «двунаправленная нейронная сеть-кодировщик») — нейронная сеть для создания модели языка.

Чтобы машины воспринимали слова, картинки или аудио, их преобразовывают в векторный вид. Когда работают с текстом, его тоже переводят в векторный формат, или векторные представления. Частный случай этих представлений — *word embeddings* (англ. «слова-вложения»; «эмбеддинги»). Работают они так: сложная структура (текст) вкладывается в более простую — вектор.
Векторы-эмбеддинги содержат данные о соотношении разных слов и их свойствах. Привычное понимание свойства слова, его смысла и контекста справедливо и для машинного обучения. 

Чтобы не создавать эмбеддинги слишком долго, возьмём из датасета выборку из 1000 случайных элементов.

In [9]:
data_sample = data.sample(
    n=1000, random_state=RANDOM_STATE).reset_index(drop=True)

Проверим, сохранилось ли соотношение количества классов в таргете сэмплированной выборки.

In [10]:
data_sample['toxic'].value_counts()

0    890
1    110
Name: toxic, dtype: int64

<a name="33"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
### 3.3. Разделение сэмпла на выборки

Разделим сэмплированную выборку `data_sample` на обучающую и тестовую выборки в соотношении 4:1.

In [11]:
data_sample_train, data_sample_test, y_train, y_test = train_test_split(
    data_sample['text'], data_sample['toxic'], test_size=TEST_SIZE, 
    stratify=data_sample['toxic'], random_state=RANDOM_STATE)

print('Размеры выборок:')
print(f"train - {len(data_sample_train)} \
- {len(data_sample_train)/len(data_sample['text']):.0%}")

print(f"test - {len(data_sample_test)} \
- {len(data_sample_test)/len(data_sample['text']):.0%}")

Размеры выборок:
train - 800 - 80%
test - 200 - 20%


<a name="34"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
### 3.4. Получение эмбеддингов (с *BERT*)

Переведём текст комментариев в векторные представления (эмбеддинги) **на базе нейронной сети *BERT***. Для этого используем модель [unitary/toxic-bert](https://huggingface.co/unitary/toxic-bert/tree/main), предобученную специально для идентификации токсичных комментариев. У *BERT* есть собственный токенизатор. Лемматизация для *BERT* не требуется.

Решим задачу проекта на *PyTorch* (англ. «факел для Python»). Библиотека `torch` применяется в задачах обработки естественного текста и компьютерного зрения. А нам нужна для работы с моделью *BERT*, которая находится в библиотеке `transformers` (англ. «трансформеры»).

Напишем функцию `get_embeddings()` для получения эмбеддингов.

In [12]:
def get_embeddings(data_sample): 
        
    # Инициализируем токенизатор как объект класса BertTokenizer().
    # Передадим ему аргумент vocab_file — это файл со словарём, 
    # на котором обучалась модель. Он может быть, например, 
    # в текстовом формате (txt). Зададим длину текстов max_length=512.
    try:
        tokenizer = transformers.BertTokenizer(vocab_file=r'C:/Users/lorad/OneDrive/'
                                               'Documents/Моя папка/Data Science/Курсы/'
                       'Яндекс.Практикум. Специалист по Data Science/Проектная работа/'
                       '13. Машинное обучение для текстов/vocab.txt', max_length=512)
    except:
        tokenizer = transformers.BertTokenizer(vocab_file=r'D:/Юлия/Data Science/Курсы/'
                       'Яндекс.Практикум. Специалист по Data Science/Проектная работа/'
                       '13. Машинное обучение для текстов/vocab.txt', max_length=512)
    
    
    # Токенизируем комментарии:    
    
    # Преобразуем текст в номера токенов из словаря методом encode() 
    # (англ. «закодировать»). Для корректной работы модели мы указали 
    # аргумент add_special_tokens=True (англ. «добавить специальные токены»).
    # Это значит, что к любому преобразуемому тексту 
    # добавляется токен начала (101) и токен конца текста (102). 
    tokenized = data_sample.apply(
        (lambda x: tokenizer.encode(x, add_special_tokens=True)))
    
    # Применим метод padding (англ. «отступ»), чтобы после токенизации 
    # длины исходных текстов в корпусе были равными. 
    # Только при таком условии будет работать модель BERT.
    # Стандартная длина вектора будет max_len=512. 
    # Остальные векторы дополним нулями.
    max_len = 512
    padded = np.array([i + [0]*(max_len - len(i)) if len(i)<512 \
                       else i[:512] for i in tokenized.values])

    
    # Cоздадим маску для выделения важных токенов:
    
    # Поясним модели, что нули не несут значимой информации. Это нужно
    # для компоненты модели, которая называется «внимание» (англ. attention).
    # Отбросим эти токены и «создадим маску» для действительно важных 
    # токенов, то есть укажем нулевые и не нулевые значения.
    attention_mask = np.where(padded != 0, 1, 0)

    # отобразим размеры преобразованных данных
    print(f"tokenized shape: {tokenized.shape}")
    print(f"padded shape: {padded.shape}")
    print(f"attention_mask shape: {attention_mask.shape}")
    
    
    # Инициализируем конфигурацию BertConfig (англ. Bert Configuration). 
    # В качестве аргумента передадим ей JSON-файл с описанием настроек модели. 
    # JSON (англ. JavaScript Object Notation, «объектная запись JavaScript») — 
    # это организованный по ключам поток цифр, букв, двоеточий и фигурных 
    # скобок, который возвращает сервер при запросе. 
    try:
        config = transformers.BertConfig.from_json_file(
            r'C:/Users/lorad/OneDrive/Documents/Моя папка/Data Science/Курсы/'
            'Яндекс.Практикум. Специалист по Data Science/Проектная работа/'
            '13. Машинное обучение для текстов/config.json')
    except:
        config = transformers.BertConfig.from_json_file(
            r'D:/Юлия/Data Science/Курсы/'
            'Яндекс.Практикум. Специалист по Data Science/Проектная работа/'
            '13. Машинное обучение для текстов/config.json')
    
    # Затем инициализируем саму модель класса BertModel. 
    # Передадим ей файл с предобученной моделью и конфигурацией.
    try:
        model = transformers.BertModel.from_pretrained(
            r'C:/Users/lorad/OneDrive/Documents/Моя папка/Data Science/Курсы/'
            'Яндекс.Практикум. Специалист по Data Science/Проектная работа/'
            '13. Машинное обучение для текстов/pytorch_model.bin', config=config)
    except:
        model = transformers.BertModel.from_pretrained(
            r'D:/Юлия/Data Science/Курсы/'
            'Яндекс.Практикум. Специалист по Data Science/Проектная работа/'
            '13. Машинное обучение для текстов/pytorch_model.bin', config=config)
                
    
    # Модель BERT создаёт эмбеддинги батчами. Чтобы хватило оперативной памяти, 
    # сделаем размер батча небольшим: создадим эмбеддинги батчами по 20 текстов.
    batch_size = 20
    # Сделаем цикл по батчам. Отображать прогресс будет функция notebook().
    # Cделаем пустой список для хранения эмбеддингов твитов.
    embeddings = []
    for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
            # Преобразуем данные в формат тензоров (англ. tensor) — многомерных  
            # векторов в библиотеке torch. Тип данных LongTensor (англ. «длинный тензор») 
            # хранит числа в «длинном формате», то есть выделяет на каждое число 64 бита.
            # Преобразуем данные.
            batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
            # Преобразуем маску.
            attention_mask_batch = torch.LongTensor(
                attention_mask[batch_size*i:batch_size*(i+1)])

            
            # Получим эмбеддинги для батча:
            
            # Для ускорения вычисления функцией no_grad() (англ. no gradient, 
            # «нет градиента») в библиотеке torch укажем, что градиенты не нужны -
            # модель BERT обучать не будем.
            with torch.no_grad():
                # чтобы получить эмбеддинги для батча, передадим модели данные и маску
                batch_embeddings = model(batch, attention_mask=attention_mask_batch)

            # Из полученного тензора извлечём нужные элементы и добавим в список 
            # всех эмбеддингов. Преобразуем элементы методом numpy() к типу numpy.array.
            embeddings.append(batch_embeddings[0][:,0,:].numpy())
    
    # cоберём и выведем все эмбеддинги в матрицу признаков вызовом функции concatenate()
    return np.concatenate(embeddings)

Применим функцию `get_embeddings()` к обучающей выборке `data_sample_train`, получим эмбеддинги `X_train` - признаки для обучения моделей.

In [13]:
%%time

X_train = get_embeddings(data_sample_train)
print(f"features shape: {X_train.shape}")

tokenized shape: (800,)
padded shape: (800, 512)
attention_mask shape: (800, 512)


Some weights of the model checkpoint at C:/Users/lorad/OneDrive/Documents/Моя папка/Data Science/Курсы/Яндекс.Практикум. Специалист по Data Science/Проектная работа/13. Машинное обучение для текстов/pytorch_model.bin were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/40 [00:00<?, ?it/s]

features shape: (800, 768)
CPU times: total: 2h 26min 13s
Wall time: 18min 49s


Применим функцию `get_embeddings()` к тестовой выборке `data_sample_test`, получим эмбеддинги `X_test` - признаки для тестирования лучшей модели.

In [14]:
%%time

X_test = get_embeddings(data_sample_test)
print(f"features shape: {X_test.shape}")

tokenized shape: (200,)
padded shape: (200, 512)
attention_mask shape: (200, 512)


Some weights of the model checkpoint at C:/Users/lorad/OneDrive/Documents/Моя папка/Data Science/Курсы/Яндекс.Практикум. Специалист по Data Science/Проектная работа/13. Машинное обучение для текстов/pytorch_model.bin were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/10 [00:00<?, ?it/s]

features shape: (200, 768)
CPU times: total: 31min 57s
Wall time: 6min 49s


<a name="35"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
### 3.5. Вывод

В разделе [**Подготовка данных**](#3.-Подготовка-данных) были выполнены следующие задачи:
1. данные изучены;
2. данные сэмплированы: из датасета взята выборка `data_sample` из 1000 случайных элементов;
3. сэмплированная выборка разделена на обучающую и тестовую выборки в соотношении 4:1;
4. получены эмбеддинги.


В результате выполнения задач этого раздела было выявлено следующее:
1. пропусков в данных нет;
2. типы данных соответствуют их содержанию;
3. датасет имеет большой размер: содержит 159 292 текстовых комментария;
4. явных дубликатов нет;
5. наблюдается сильный дисбаланс классов в таргете.

**В проекте решается задача бинарной классификации.**

Таким образом, данные подготовлены для обучения моделей.

<a name="4"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
## 4. Обучение и тестирование моделей

Обучим четыре модели: логистическую регрессию (*Logistic Regression*), случайный лес (*Random Forest*), градиентные бустинги *LightGBM* и *CatBoost* для задачи классификации. Для обучения логистической регрессии используем функцию `cross_val_score()`, а для последних трёх моделей применим поиск гиперпараметров с помощью функции `GridSearchCV`.

Учтём баланс классов в модели с помощью параметра `class_weight`.

<a name="41"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
### 4.1. Функция для обучения моделей

Напишем функцию `fit_model()` для обучения моделей `RandomForestClassifier`, `LGBMClassifier` и `CatBoostClassifier` и вычисления метрики качества *F1*. Метрика *F1* лучшей модели будет выводиться как `model.best_score_`. 

Используем разделение на фолды с помощью *KFold*. Зададим параметры для кроссвалидации: `n_splits` - количество фолдов.

In [15]:
kfold = KFold(n_splits=3, random_state=RANDOM_STATE, shuffle=True)

In [16]:
def fit_model(estimator, param_grid, X_train, y_train):
    model = GridSearchCV(estimator=estimator,
                         param_grid=param_grid,
                         cv=kfold,
                         scoring='f1')

    model.fit(X_train, y_train)
    best_f1 = round(model.best_score_, 2)

    print(f"Best F1: {best_f1}")
    print(f"Best params: {model.best_params_}")

    return model.best_estimator_, best_f1

<a name="42"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
### 4.2. `LogisticRegression`

Классифицируем обучающие данные с помощью модели **логистической регрессии *Logistic Regression*** - `LogisticRegression`. Используем функцию `cross_val_score()` для автоматизации предварительных преобразований данных перед обучением модели.

In [17]:
%%time

model_lr = LogisticRegression(random_state=RANDOM_STATE, 
                              class_weight='balanced')
model_lr.fit(X_train, y_train)

f1_lr = cross_val_score(model_lr, X_train, y_train,
                        scoring='f1', cv=4).mean()

print('F1:', f'{f1_lr:.2f}')

F1: 0.93
CPU times: total: 1.06 s
Wall time: 434 ms


Модель логистической регрессии **`LogisticRegression`** на обучающей выборке имеет следующее значение метрики оценки качества:
- ***F1 = 0.93***

<a name="43"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
### 4.3. `RandomForestClassifier`

Построим модель **случайного леса *Random Forest*** - `RandomForestClassifier`.

In [18]:
%%time

rf_estimator = RandomForestClassifier(random_state=RANDOM_STATE, 
                                      class_weight='balanced')

rf_param_grid = {
    # количество деревьев
    'n_estimators': list(range(60, 121, 30)),
    # максимальная глубина дерева
    'max_depth': list(range(2, 13, 5)),
}

rf_best_model = fit_model(estimator=rf_estimator,
                          param_grid=rf_param_grid,
                          X_train=X_train,
                          y_train=y_train)

Best F1: 0.94
Best params: {'max_depth': 7, 'n_estimators': 60}
CPU times: total: 4.89 s
Wall time: 4.89 s


Лучшая модель случайного леса **`RandomForestClassifier`** на обучающей выборке имеет следующее значение метрики оценки качества:
- ***F1 = 0.94*** 

при следующих параметрах:
- глубина дерева: `max_depth` = 7
- количество деревьев: `n_estimators` = 60

<a name="44"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
### 4.4. `LGBMClassifier`

Построим модель **градиентного бустинга *LightGBM*** - `LGBMClassifier`.

In [19]:
%%time

lgbm_estimator = LGBMClassifier(random_state=RANDOM_STATE, 
                                class_weight='balanced')

lgbm_param_grid = {
    # количество деревьев (итераций)
    "n_estimators": range(25, 101, 25), 
    # максимальная глубина дерева
    "max_depth": range(5, 16, 5),
    # коэффициент скорости обучения (размер шага градиентного спуска)
    'learning_rate': [0.15, 0.2, 0.25]
}

lgbm_best_model = fit_model(estimator=lgbm_estimator,
                            param_grid=lgbm_param_grid,
                            X_train=X_train,
                            y_train=y_train)

Best F1: 0.94
Best params: {'learning_rate': 0.15, 'max_depth': 5, 'n_estimators': 100}
CPU times: total: 6min 49s
Wall time: 27.8 s


Лучшая модель градиентного бустинга **`LGBMClassifier`** на обучающей выборке имеет следующее значение метрики оценки качества:
- ***F1 = 0.94***

при следующих параметрах:
- глубина дерева: `max_depth` = 5
- количество деревьев: `n_estimators` = 100
- коэффициент скорости обучения (размер шага градиентного спуска): `learning_rate` = 0.15

<a name="45"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
### 4.5. `CatBoostClassifier`

Построим модель **градиентного бустинга *CatBoost*** - `CatBoostClassifier`.

`CatBoostClassifier` создает временные файлы для обучения. Чтобы это предотвратить, и чтобы не появлялась ошибка о том, что по указанными путям у `CatBoostClassifier` нет доступа к созданию файлов, укажем в модели параметр `allow_writing_files=False`.

In [20]:
%%time

classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', 
                               classes=classes, 
                               y=y_train)
class_weights = dict(zip(classes, weights))

catboost_estimator = CatBoostClassifier(random_state=RANDOM_STATE, 
                                        verbose=False, 
                                        class_weights=class_weights,
                                        allow_writing_files=False)


catboost_param_grid = {
    # количество итераций
    "iterations": range(100, 201, 50),
    # глубина дерева
    "depth": range(2, 7, 2),
    # коэффициент скорости обучения (размер шага градиентного спуска)
    "learning_rate": [0.1, 0.15, 0.2],
}

catboost_best_model = fit_model(estimator=catboost_estimator,
                                param_grid=catboost_param_grid,
                                X_train=X_train,
                                y_train=y_train)

Best F1: 0.92
Best params: {'depth': 4, 'iterations': 200, 'learning_rate': 0.15}
CPU times: total: 34min 42s
Wall time: 4min 46s


Лучшая модель градиентного бустинга **`CatBoostClassifier`** на обучающей выборке имеет следующее значение метрики оценки качества:
- ***F1 = 0.92*** 

при следующих параметрах:
- глубина дерева: `depth` = 4
- количество итераций: `iterations` = 200
- коэффициент скорости обучения (размер шага градиентного спуска): `learning_rate` = 0.15

<a name="46"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
### 4.6. Сравнение моделей

Выведем значения метрики *F1* разных моделей на обучающей выборке в виде таблицы `table`.

In [21]:
table = pd.DataFrame([
     ['LogisticRegression', f'{f1_lr:.2f}'],
     ['RandomForestClassifier', f'{rf_best_model[1]:.2f}'],
     ['LGBMClassifier', f'{lgbm_best_model[1]:.2f}' ],
     ['CatBoostClassifier', f'{catboost_best_model[1]:.2f}']
    ],
columns=['model', 'F1'])

print('F1 для разных моделей')
table

F1 для разных моделей


,model,F1
0,LogisticRegression,0.93
1,RandomForestClassifier,0.94
2,LGBMClassifier,0.94
3,CatBoostClassifier,0.92


В качестве лучшей выберем модель **градиентного бустинга `LGBMClassifier`** со значением метрики ***F1 = 0.94***.

<a name="47"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
### 4.7. Тестирование лучшей модели

Проверим на тестовой выборке качество выбранной модели **градиентного бустинга `LGBMClassifier`**.

In [22]:
f1_lgbm_test = f1_score(y_test, lgbm_best_model[0].predict(X_test))
print('F1 на тестовой выборке:', f'{f1_lgbm_test:.2f}')

F1 на тестовой выборке: 0.95


Лучшая модель градиентного бустинга **`LGBMClassifier`** на тестовой выборке имеет следующее значение метрики оценки качества:
- ***F1 = 0.95***

Значение метрики *F1* на тестовой выборке превышает 0.75, что соответствует изначальному требованию в условии задачи проекта.

<a name="48"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
### 4.8. Вывод

В разделе [**Обучение и тестирование моделей**](#4.-Обучение-и-тестирование-моделей) были выполнены следующие задачи:
1. написана функция `fit_model()` для обучения и вычисления метрики *F1* для моделей с использованием `GridSearchCV`;
2. обучено четыре модели: `LogisticRegression`, `RandomForestClassifier`, `LGBMClassifier` и `CatBoostClassifier` с различными гиперпараметрами;
3. качесто лучшей модели проверено на тестовой выборке.

В результате выполнения задач этого раздела было выявлено следующее:
1. В качестве лучшей выбрана модель **`LGBMClassifier`**, у которой на обучающей выборке значение метрики оценки качества ***F1 = 0,94***.
2. Лучшая модель градиентного бустинга **`LGBMClassifier`** на тестовой выборке имеет следующее значение метрики оценки качества:
- ***F1 = 0.95***

Значение метрики *F1* на тестовой выборке превышает 0.75, что соответствует изначальному требованию в условии задачи проекта.

<a name="5"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
## 5. Выводы

Проведено исследование с целью построения модели машинного обучения, которая поможет классифицировать комментарии на позитивные и негативные. Поставленная задача была решена **на базе нейронной сети *BERT***.

Результаты исследования позволят магазину искать токсичные комментарии и отправлять их на модерацию.

Входные данные: набор данных с разметкой о токсичности правок.

В ходе исследования удалось получить следующие результаты **на обучающей выборке**:


1. Модель логистической регрессии **`LogisticRegression`** имеет следующее значение метрики оценки качества:
    - ***F1 = 0.93***


2. Лучшая модель случайного леса **`RandomForestClassifier`** имеет следующее значение метрики оценки качества:
    - ***F1 = 0.94*** 

   при следующих параметрах:
    - глубина дерева: `max_depth` = 7
    - количество деревьев: `n_estimators` = 60


3. Лучшая модель градиентного бустинга **`LGBMClassifier`** имеет следующее значение метрики оценки качества:
    - ***F1 = 0.94***

   при следующих параметрах:
    - глубина дерева: `max_depth` = 5
    - количество деревьев: `n_estimators` = 100
    - коэффициент скорости обучения (размер шага градиентного спуска): `learning_rate` = 0.15


4. Лучшая модель градиентного бустинга **`CatBoostClassifier`** имеет следующее значение метрики оценки качества:
    - ***F1 = 0.92*** 

   при следующих параметрах:
    - глубина дерева: `depth` = 4
    - количество итераций: `iterations` = 200
    - коэффициент скорости обучения (размер шага градиентного спуска): `learning_rate` = 0.15
      
      
Исходя из полученных результатов, можно сделать следующие **выводы**:


1. В качестве лучшей модели выбрана:
   - модель **градиентного бустинга `LGBMClassifier`**, которая **на тестовой выборке** имеет следующее значение метрики оценки качества:
    - ***F1 = 0.95*** 

   
2. Для выбранной лучшей модели значение метрики качества *F1* превышает 0.75, что соответсвует изначальному требованию в условии задачи проекта.
   
   
**Общие рекомендации:**

Магазину можно рекомендовать использовать полученную модель **`LGBMClassifier`** в качестве инструмента, который будет искать токсичные комментарии и отправлять их на модерацию.